In [1]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [2]:
data = pd.read_csv('daily-website-visitors.csv')


In [3]:
data

,Row,Day,Day.Of.Week,Date,Page.Loads,Unique.Visits,First.Time.Visits,Returning.Visits
0,1,Sunday,1,9/14/2014,2146,1582,1430,152
1,2,Monday,2,9/15/2014,3621,2528,2297,231
2,3,Tuesday,3,9/16/2014,3698,2630,2352,278
3,4,Wednesday,4,9/17/2014,3667,2614,2327,287
4,5,Thursday,5,9/18/2014,3316,2366,2130,236
...,...,...,...,...,...,...,...,...
2162,2163,Saturday,7,8/15/2020,2221,1696,1373,323
2163,2164,Sunday,1,8/16/2020,2724,2037,1686,351
2164,2165,Monday,2,8/17/2020,3456,2638,2181,457
2165,2166,Tuesday,3,8/18/2020,3581,2683,2184,499


In [4]:
data = data.filter(['Page.Loads'])


In [5]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Page.Loads
1167,2658
1168,2391
1169,3202
1170,4349
1171,4886
...,...
2162,2221
2163,2724
2164,3456
2165,3581


In [6]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [7]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [8]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [9]:
start_time = time.time()

In [10]:

# Tạo mô hình CNN
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(size_window, 1)))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
for i in range(7): 
    model.add(Dense(units=100, activation='relu')) 
    model.add(Dense(units=1))

# Compile mô hình với Adam optimizer và Mean Squared Error loss
model.compile(loss='mse', optimizer='adam')


In [11]:
history = model.fit(X_train, y_train, epochs=10000, batch_size=16)

Epoch 1/10000
50/50 [==============================] - 3s 2ms/step - loss: 0.1207
Epoch 2/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0444
Epoch 3/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0266
Epoch 4/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0131
Epoch 5/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0113
Epoch 6/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0107
Epoch 7/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0088
Epoch 8/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0088
Epoch 9/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 10/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 11/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 12/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0063
Epoch 13/1000

50/50 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 100/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 101/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 102/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 103/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 104/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 105/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 106/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 107/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0021
Epoch 108/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 109/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 110/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 11

50/50 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 197/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 198/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 199/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 200/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 201/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0013
Epoch 202/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 203/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 204/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 205/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 206/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 207/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 20

50/50 [==============================] - 0s 2ms/step - loss: 7.5531e-04
Epoch 292/10000
50/50 [==============================] - 0s 2ms/step - loss: 8.2579e-04
Epoch 293/10000
50/50 [==============================] - 0s 2ms/step - loss: 7.3849e-04
Epoch 294/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 295/10000
50/50 [==============================] - 0s 2ms/step - loss: 9.5729e-04
Epoch 296/10000
50/50 [==============================] - 0s 2ms/step - loss: 7.7113e-04
Epoch 297/10000
50/50 [==============================] - 0s 2ms/step - loss: 8.9010e-04
Epoch 298/10000
50/50 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 299/10000
50/50 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 300/10000
50/50 [==============================] - 0s 2ms/step - loss: 8.6975e-04
Epoch 301/10000
50/50 [==============================] - 0s 2ms/step - loss: 8.4269e-04
Epoch 302/10000
50/50 [==============================] - 0s 

50/50 [==============================] - 0s 3ms/step - loss: 6.0673e-04
Epoch 385/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.3901e-04
Epoch 386/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.7586e-04
Epoch 387/10000
50/50 [==============================] - 0s 5ms/step - loss: 5.4503e-04
Epoch 388/10000
50/50 [==============================] - 0s 5ms/step - loss: 5.0078e-04
Epoch 389/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.4981e-04
Epoch 390/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.1553e-04
Epoch 391/10000
50/50 [==============================] - 0s 4ms/step - loss: 7.1151e-04
Epoch 392/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.6430e-04
Epoch 393/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.7336e-04
Epoch 394/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.8357e-04
Epoch 395/10000
50/50 [=========================

...

50/50 [==============================] - 0s 3ms/step - loss: 1.6650e-05
Epoch 9591/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.5742e-05
Epoch 9592/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.0424e-05
Epoch 9593/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.8337e-05
Epoch 9594/10000
50/50 [==============================] - 0s 3ms/step - loss: 9.5443e-06
Epoch 9595/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.5499e-06
Epoch 9596/10000
50/50 [==============================] - 0s 3ms/step - loss: 7.1431e-06
Epoch 9597/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.8071e-06
Epoch 9598/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.8956e-06
Epoch 9599/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.6595e-06
Epoch 9600/10000
50/50 [==============================] - 0s 3ms/step - loss: 2.6943e-06
Epoch 9601/10000
50/50 [==============

50/50 [==============================] - 0s 3ms/step - loss: 1.5581e-05
Epoch 9683/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.2175e-05
Epoch 9684/10000
50/50 [==============================] - 0s 3ms/step - loss: 8.1738e-06
Epoch 9685/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.3787e-06
Epoch 9686/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.4168e-06
Epoch 9687/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.4471e-06
Epoch 9688/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.2069e-06
Epoch 9689/10000
50/50 [==============================] - 0s 3ms/step - loss: 2.2458e-06
Epoch 9690/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.8126e-06
Epoch 9691/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.7044e-06
Epoch 9692/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.6034e-06
Epoch 9693/10000
50/50 [==============

50/50 [==============================] - 0s 3ms/step - loss: 3.2846e-05
Epoch 9775/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.4704e-05
Epoch 9776/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.6487e-05
Epoch 9777/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.1965e-05
Epoch 9778/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.6554e-05
Epoch 9779/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.1681e-05
Epoch 9780/10000
50/50 [==============================] - 0s 2ms/step - loss: 6.9089e-06
Epoch 9781/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.7175e-06
Epoch 9782/10000
50/50 [==============================] - 0s 3ms/step - loss: 2.5323e-06
Epoch 9783/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.9219e-06
Epoch 9784/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.2059e-06
Epoch 9785/10000
50/50 [==============

50/50 [==============================] - 0s 3ms/step - loss: 6.9373e-05
Epoch 9867/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.1226e-04
Epoch 9868/10000
50/50 [==============================] - 0s 3ms/step - loss: 5.6344e-05
Epoch 9869/10000
50/50 [==============================] - 0s 3ms/step - loss: 2.1666e-05
Epoch 9870/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.0182e-05
Epoch 9871/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.9166e-06
Epoch 9872/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.1732e-06
Epoch 9873/10000
50/50 [==============================] - 0s 3ms/step - loss: 3.0607e-06
Epoch 9874/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.9729e-06
Epoch 9875/10000
50/50 [==============================] - 0s 3ms/step - loss: 2.7380e-06
Epoch 9876/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.4017e-06
Epoch 9877/10000
50/50 [==============

50/50 [==============================] - 0s 3ms/step - loss: 6.0793e-07
Epoch 9959/10000
50/50 [==============================] - 0s 3ms/step - loss: 6.3447e-07
Epoch 9960/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.7833e-06
Epoch 9961/10000
50/50 [==============================] - 0s 4ms/step - loss: 1.4324e-06
Epoch 9962/10000
50/50 [==============================] - 0s 4ms/step - loss: 2.6986e-06
Epoch 9963/10000
50/50 [==============================] - 0s 3ms/step - loss: 4.7176e-06
Epoch 9964/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.3251e-05
Epoch 9965/10000
50/50 [==============================] - 0s 3ms/step - loss: 1.0476e-05
Epoch 9966/10000
50/50 [==============================] - 0s 3ms/step - loss: 8.5409e-06
Epoch 9967/10000
50/50 [==============================] - 0s 3ms/step - loss: 7.9376e-06
Epoch 9968/10000
50/50 [==============================] - 0s 3ms/step - loss: 9.0416e-06
Epoch 9969/10000
50/50 [==============

In [12]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

6/6 [==============================] - 0s 1ms/step
Training time: 1522.300541639328 seconds


In [13]:
import h5py
from keras.models import save_model

# Lưu mô hình CNN vào file model.h5
save_model(model, 'Daily_CNN_82.h5')

# Tạo file H5 mới
with h5py.File('Daily_CNN_82_data.h5', 'w') as f:
    # Lưu giá trị y_pred
    f.create_dataset('y_test', data=y_test)
    
    # Lưu giá trị X_test
    f.create_dataset('X_test', data=X_test)
    
    # Lưu giá trị training_time
    f.create_dataset('training_time', data=training_time)
